In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from yellowbrick.text import FreqDistVisualizer
from pathlib import Path
nlp=spacy.load('en_core_web_sm')
from textblob import TextBlob, Word, Blobber

from sb_utils import save_file


from surprise import accuracy, Dataset, SVD 
from surprise import Reader
from surprise.model_selection import cross_validate, train_test_split, LeaveOneOut, ShuffleSplit 
from surprise.model_selection import RandomizedSearchCV,GridSearchCV
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline
from surprise.model_selection import KFold

In [2]:
#full dataset
#wine_df2 = pd.read_csv('C:/Users/jwatki8/Documents/Data Science Program information/Capstone project 3/data/wine_df.csv')

In [3]:
wine_df = pd.read_csv('C:/Users/jwatki8/Documents/Data Science Program information/Capstone project 3/data/wine_df_S.csv')

In [4]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 50 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   RatingID                                150000 non-null  int64  
 1   UserID                                  150000 non-null  int64  
 2   WineID                                  150000 non-null  int64  
 3   Vintage                                 150000 non-null  object 
 4   Rating                                  150000 non-null  float64
 5   Date                                    150000 non-null  object 
 6   rating_mean                             150000 non-null  float64
 7   rater_count                             150000 non-null  int64  
 8   rater_mean                              150000 non-null  float64
 9   WineName                                150000 non-null  object 
 10  Type                                    1500

Since our recommender system will be based on user ratings of wine, let’s start by parsing out the user id, wine id and rating.

In [5]:
rec_base=wine_df[['UserID', 'WineID','Rating']]

In [6]:
rec_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   UserID  150000 non-null  int64  
 1   WineID  150000 non-null  int64  
 2   Rating  150000 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.4 MB


In [7]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rec_base[['UserID', 'WineID','Rating']], reader)
anti_set = data.build_full_trainset().build_anti_testset()

In [ ]:
#Train test split

I would like to examine a few different types of splits for the data. First let’s take a look at a traditional train test split.

In [8]:
trainset, testset = train_test_split(data, test_size=0.25)

In [9]:
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.4844
MAE:  0.3548


0.35483597033810665

In [10]:
#trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo2 = KNNBaseline()

# Train the algorithm on the trainset, and predict ratings for the testset
algo2.fit(trainset)
predictions = algo2.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.4820
MAE:  0.3517


0.35170036676907257

In [21]:
params = {'n_factors': [25, 50, 100],
         'reg_all': [.02, .05, .1],'lr_all': [0.002, 0.005]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
g_s_svd.fit(data)

# best RMSE score
print(g_s_svd.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(g_s_svd.best_params["rmse"])

0.47849875860334234
{'n_factors': 25, 'reg_all': 0.05, 'lr_all': 0.005}


In [24]:
algo = SVD(n_factors=25,reg_all=0.05,lr_all=0.005)

cross_validate(algo, data, measures=["RMSE", "MAE"], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.4841  0.4839  0.4796  0.4825  0.0021  
MAE (testset)     0.3544  0.3536  0.3526  0.3535  0.0008  
Fit time          0.93    0.92    0.93    0.92    0.00    
Test time         0.81    0.36    0.63    0.60    0.18    


{'test_rmse': array([0.48414091, 0.48387144, 0.47961521]),
 'test_mae': array([0.3544013 , 0.35359566, 0.3525615 ]),
 'fit_time': (0.9255781173706055, 0.9209592342376709, 0.9267685413360596),
 'test_time': (0.8054673671722412, 0.35544347763061523, 0.6262245178222656)}

In [24]:
algo = SVD(n_factors=25,reg_all=0.05)

cross_validate(algo, data, measures=["RMSE", "MAE"], cv=lo, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.5076  0.4947  0.4953  0.4992  0.0059  
MAE (testset)     0.3668  0.3605  0.3606  0.3626  0.0029  
Fit time          1.24    1.18    1.14    1.19    0.04    
Test time         0.13    0.07    0.10    0.10    0.02    


{'test_rmse': array([0.50759131, 0.49473782, 0.49526982]),
 'test_mae': array([0.36676033, 0.36053751, 0.36062853]),
 'fit_time': (1.2448101043701172, 1.1755585670471191, 1.138174295425415),
 'test_time': (0.13187146186828613, 0.07233285903930664, 0.09816288948059082)}

In [25]:
#start using full data set here
dataset = data.build_full_trainset()
svd = SVD(n_factors=25,reg_all=0.05,lr_all=0.005)
svd.fit(dataset)
#svd.predict(3, 6)
svd.predict(uid=1061261, iid=154991)

Prediction(uid=1061261, iid=154991, r_ui=None, est=3.82003, details={'was_impossible': False})

In [ ]:
#Shuffle split cross validation verbose=True

Next I would like to see the results of a Shuffle split.

In [10]:
# define a cross-validation iterator
ss = ShuffleSplit(n_splits=3,random_state=22)

algo = SVD()

for trainset, testset in ss.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions)
    accuracy.mae(predictions)

RMSE: 0.4835
MAE:  0.3560
RMSE: 0.4838
MAE:  0.3560
RMSE: 0.4829
MAE:  0.3554


In [ ]:
#leave one out cross validation

I also want to examine a Leave One Out split.

In [23]:
# define a cross-validation iterator
lo = LeaveOneOut(n_splits=3,random_state=22)

algo = SVD()

for trainset, testset in lo.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions)
    accuracy.mae(predictions)

RMSE: 0.5105
MAE:  0.3703
RMSE: 0.4996
MAE:  0.3658
RMSE: 0.5004
MAE:  0.3660


After examining the models, it looks like the ShuffleSplit model gives the best results. It also makes the most sense to use the Root Mean Squared error to evaluate the model because of the size of our dataset.

In [20]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rec_base[['UserID', 'WineID','Rating']], reader)


In [11]:
ss = ShuffleSplit(random_state=22)
algo = SVD()
#algo = SVD(n_epochs=5,lr_all=0.002,reg_all=0.4)

In [12]:
param_grid = {"n_epochs": [5, 10]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=ss)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.4921812387993036
{'n_epochs': 10}


In [15]:
param_grid = {"n_epochs": [10], "lr_all": [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=ss)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.4920504484251108
{'n_epochs': 10, 'lr_all': 0.005}


In [16]:
param_grid = {"n_epochs": [10], "lr_all": [0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=ss)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.5029384831051555
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [17]:
param_distributions = {"n_epochs": [5, 10]}
rcv = RandomizedSearchCV(SVD, param_distributions, measures=["rmse", "mae"], cv=ss)

rcv.fit(data)

# best RMSE score
print(rcv.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(rcv.best_params["rmse"])

ValueError: Cannot take a larger sample than population when 'replace=False'

In [29]:
param_distributions = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
rcv = RandomizedSearchCV(SVD, param_distributions, measures=["rmse", "mae"], cv=ss)

#rcv.fit(data)

# best RMSE score
#print(rcv.best_score["rmse"])

# combination of parameters that gave the best RMSE score
#print(rcv.best_params["rmse"])

ValueError: Cannot take a larger sample than population when 'replace=False'

In [19]:
#svd.predict(uid=1061261, iid=154991)
#pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Prediction(uid=1061261, iid=154991, r_ui=None, est=4.321918823637014, details={'was_impossible': False})

In [ ]:
#svd = SVD(verbose=True, n_epochs=10)
#cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
#cross_validate(algo, data, measures=["RMSE", "MAE"], cv=3, verbose=True)

In [11]:
#svd = SVD(verbose=True, n_epochs=10)
#cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.6100  0.6098  0.6093  0.6097  0.0003  
MAE (testset)     0.4445  0.4444  0.4441  0.4444  0.0002  
Fit time          148.34  156.56  158.67  154.52  4.45    
Test time         104.62  108.54  110.32  107.83  2.38    


{'test_rmse': array([0.61004689, 0.60977692, 0.6093211 ]),
 'test_mae': array([0.4445458 , 0.4444005 , 0.44413388]),
 'fit_time': (148.34153485298157, 156.55779576301575, 158.66535186767578),
 'test_time': (104.61968541145325, 108.53983163833618, 110.32388472557068)}